In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
from pyquation import Equation, Variable
from sympy import *
from sympy.physics.units import *
from sympy import S

In [41]:
F = Variable(force, newtons)
M = Variable(mass, kilogram)
A = Variable(acceleration, meters/second**2, 'A')

In [42]:
assert F.name == 'F'
assert F.full_name == 'force'
assert F.all_names == ('F', 'force')
assert A.name == 'A'
assert A.all_names == ('A', 'acceleration')

In [43]:
nlaw = Equation(F, M*A, 'This is Newtons 2nd law of motion')

# 1 less var
Equation.show_solved_term = False
assert nlaw(F=4*newton, M=2*kilogram) == 2*meters/second**2

# show_solved_term
Equation.show_solved_term = True
assert nlaw(F=4*newton, M=2*kilogram) == Eq(A, 2*meters/second**2)
Equation.show_solved_term = False
assert nlaw(F=4*newton, M=2*kilogram, show_solved_term=True) == Eq(A, 2*meters/second**2)
Equation.show_solved_term = True
assert nlaw(F=4*newton, M=2*kilogram, show_solved_term=False) == 2*meters/second**2
Equation.show_solved_term = False

# convert_to(solve(nlaw(F=4*newton, M=7*pound), dict=True)[0][A], A.default_unit)

# same number of vars
assert nlaw(F=4*newton, M=2*kilogram, A=2*meters/second**2)
assert not nlaw(F=4*newton, M=2*kilogram, A=3*meters/second**2)

# Custom units works
assert nlaw(F=4*newton, M=2*kilogram, units=feet/second**2) == S(2500)/381 * feet/second**2

# Too few vars works
assert nlaw(F=4*newton) == Equation(4*newton, A*M)
assert nlaw(M=2*kilogram, units=gram) == Equation(F, 2000*A*gram)

AssertionError: 

In [ ]:
nlaw(A=9.81*meters/second**2, M=12*kg)

117.72*newton

In [ ]:
assert nlaw.variables == {A, F, M}

In [ ]:
assert nlaw.restructure(M) == [Equation(M, F/A)]

In [ ]:
nlaw.help()

Eq(F, A*M)

where:
	F is force in units of newton
	M is mass in units of kilogram
	A is acceleration in units of meter/second**2

This is Newtons 2nd law of motion


In [ ]:
nlaw.full_form()

Eq(F, A*M)

In [ ]:
nlaw()

Eq(F, A*M)

In [ ]:
# uncertinties with too few vars dont do anything
assert nlaw(mass=3*kg, M_unc=2, δforce=3) == nlaw(mass=3*kg)

In [ ]:
# Parameter names
assert nlaw(M=3*kg, F=2*newton) == nlaw(mass=3*kg, force=2*newton)
assert (
    nlaw(M=2*kg, F=2*newton, F_unc=1, M_unc=2) ==
    nlaw(mass=2*kg, F=2*newton, force_unc=1, mass_unc=2) ==
    nlaw(mass=2*kg, force=2*newton, unc_force=1, unc_M=2) ==
    nlaw(mass=2*kg, F=2*newton, δforce=1, δM=2)
)

In [ ]:
# TODO: tests for specifying the unit in the Equation call, and also making sure the uncertainty is correct (in the correct units)

nlaw(A=2*meters/second**2, F=4*newtons, units=pound)
a = nlaw(A=2*meters/second**2, F=4*newtons, F_unc=1, A_unc=1, units=pound)[1].as_coeff_mul()


AttributeError: 'Symbol' object has no attribute 'default_unit'

In [ ]:
e = convert_to(3*foot, meters)
{i for i in e.atoms() if not isinstance(i, (Unit, Symbol))}
# e.replace(e.atoms(Unit, Symbol), 1, map=True)
e.as_coeff_Mul()

(1143/1250, meter)

In [ ]:
(Symbol('a')*foot).as_coeff_Mul()

(1, foot*a)

In [ ]:
e = 8*Symbol('a') - 2*Symbol('b') - Symbol('c')
e

8*a - 2*b - c

In [ ]:
e.replace(Symbol('b'), 1)

8*a - c - 2

In [ ]:
from uncertainties import ufloat as uf
from uncertainties import ufloat

In [ ]:
nlaw(F=uf(7, 3)*newton)

TypeError: can't convert an affine function (<class 'uncertainties.core.Variable'>) to float; use x.nominal_value

In [ ]:
uf(7, 3)

7.0+/-3.0

In [ ]:
lambdify(['mean', 'std'], ufloat, modules='uncertainties')

NameError: name 'ufloat' is not defined

In [ ]:
x, x1, x3 = symbols('x x1 x3')
L=Symbol("L")
# eqn=(x*(L**2))+(x1*(L*1))+(x3*(L**0))
# s = solve(eqn,L)
display(eqn)
lambdify([x, x1, x3], eqn)(ufloat(10,0.2), ufloat(8,0.01), ufloat(25,2))

L**2*x + L*x1 + x3

TypeError: can't convert an affine function (<class 'uncertainties.core.Variable'>) to float; use x.nominal_value

In [ ]:
f, m, a = symbols('f, m, a')
nlaw2 = lambdify([f, m, a], Eq(f, m*a))
# nlaw2(4, 2, 2)
# nlaw2(m=2*kilograms, a=2*meters/second**2, f=4*newtons)
nlaw2(m=2, a=2, f=4)

TypeError: _lambdifygenerated() missing 1 required positional argument: 'f'

In [ ]:
f, m, a = symbols('f, m, a')

nlaw2 = lambdify([m, a], m*a)
# nlaw2(4, 2, 2)
# nlaw2(m=2*kilograms, a=2*meters/second**2, f=4*newtons)
# convert_to(nlaw2(m=2*kilograms, a=2*meters/second**2), newtons)
nlaw2(m=uf(2, 1), a=uf(2, .5))

4.0+/-2.23606797749979

In [ ]:
from sympy.abc import x
from sympy.utilities.lambdify import implemented_function
from sympy import lambdify
f = implemented_function('f', lambda x: x+1)
lam_f = lambdify(x, f(x))
lam_f(uf(3, 2))

4.0+/-2.0

In [ ]:
f(uf(3, 1))

TypeError: can't convert an affine function (<class 'uncertainties.core.Variable'>) to float; use x.nominal_value

In [ ]:
uf(2, 1)

In [ ]:
nlaw2(4*newtons, 2*kilograms, )

TypeError: _lambdifygenerated() missing 1 required positional argument: 'a'

In [ ]:
from pycertainties import calculate, Val, uncertainty
# calculate(m*a, m=Val(3, 1), a=Val(4, 2))
uncertainty(m*a, 'm', 'a').subs(dict(m=2, a=2, δm=1, δa=S(1)/2))


TypeError: type 'Union' is not subscriptable

In [ ]:
def uncertainty(expr: Expr):
    return sqrt(sum(
        diff(expr, sym) ** 2 * Symbol("δ" + sym.name) ** 2
        for sym in expr.atoms(Symbol)
    ))

In [ ]:
uncertainty(Eq(Symbol('f'), m*a))

sqrt(δa**2*Derivative(Eq(f, a*m), a)**2 + δf**2*Derivative(Eq(f, a*m), f)**2 + δm**2*Derivative(Eq(f, a*m), m)**2)

In [ ]:
expr = solve(nlaw, F, list=True)[0]
t = sqrt(sum(
    diff(expr, sym)**2 * Symbol("δ" + sym.name)**2
    for sym in expr.atoms(Symbol)
)).subs({M:2, A:2, 'δM': 1, 'δA':1})

In [ ]:
uf(2, t)

2.0+/-2.8284271247461903

A*M

In [ ]:
# Force, mass, acceleration, velocity, radius
Matrix([[1, 1, 1, 0, 0], [0, 0, 1, ]])

Matrix([[1, 1, 1]])